In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
df_pred = pd.read_csv("./raw_data/smoking_prediction_entrega.csv")
df_pred

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar
0,27358,M,25,160,65,3.42,0.05,0.00,0.04,0.04,...,3.04,0.63,0.04,0.01,0.75,0.71,0.71,Y,0,Y
1,27364,M,30,180,80,3.46,0.04,0.01,0.04,0.04,...,4.21,0.59,0.04,0.04,0.79,1.13,1.33,Y,0,N
2,27368,M,55,165,60,3.42,0.00,0.01,0.04,0.04,...,2.04,0.63,0.04,0.01,1.08,1.29,2.00,Y,1,Y
3,27378,M,20,175,75,3.63,0.05,0.05,0.04,0.04,...,3.71,0.63,0.04,0.04,0.83,0.58,0.46,Y,0,N
4,27381,M,25,165,80,3.79,0.04,0.04,0.04,0.04,...,6.63,0.67,0.04,0.04,1.25,1.63,1.96,Y,1,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,55676,F,40,170,65,3.13,0.01,0.01,0.04,0.04,...,4.92,0.50,0.04,0.00,0.58,0.29,0.42,Y,1,Y
5688,55681,F,45,160,50,2.92,0.04,0.04,0.04,0.04,...,3.29,0.58,0.04,0.01,0.83,0.50,0.58,Y,0,Y
5689,55683,F,55,160,50,2.84,0.04,0.04,0.04,0.04,...,2.63,0.50,0.04,0.00,0.71,0.46,0.50,Y,0,N
5690,55684,M,60,165,60,3.25,0.01,0.04,0.04,0.04,...,6.08,0.59,0.04,0.00,0.83,0.79,0.75,Y,0,N


In [3]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5692 entries, 0 to 5691
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   5692 non-null   int64  
 1   gender               5692 non-null   object 
 2   age                  5692 non-null   int64  
 3   height(cm)           5692 non-null   int64  
 4   weight(kg)           5692 non-null   int64  
 5   waist(cm)            5692 non-null   float64
 6   eyesight(left)       5692 non-null   float64
 7   eyesight(right)      5692 non-null   float64
 8   hearing(left)        5692 non-null   float64
 9   hearing(right)       5692 non-null   float64
 10  systolic             5692 non-null   float64
 11  relaxation           5692 non-null   float64
 12  fasting blood sugar  5692 non-null   float64
 13  Cholesterol          5692 non-null   float64
 14  triglyceride         5692 non-null   float64
 15  HDL                  5692 non-null   f

In [4]:
temp_df = pd.DataFrame(columns=['column', 'dtype', 'unique_values'])
for col in df_pred.columns:
  coltype = df_pred[col].dtype
  temp_df = pd.concat([temp_df, pd.DataFrame({
      'column': [col],
      'dtype': [coltype],
      'unique_values': [df_pred[col].nunique()]
  })], ignore_index=True)
  
temp_df

,column,dtype,unique_values
0,ID,int64,5692
1,gender,object,2
2,age,int64,14
3,height(cm),int64,12
4,weight(kg),int64,18
5,waist(cm),float64,85
6,eyesight(left),float64,6
7,eyesight(right),float64,6
8,hearing(left),float64,2
9,hearing(right),float64,2


In [5]:
ids = df_pred["ID"]
ids

0       27358
1       27364
2       27368
3       27378
4       27381
        ...  
5687    55676
5688    55681
5689    55683
5690    55684
5691    55691
Name: ID, Length: 5692, dtype: int64

Limpieza de datos

In [6]:
cols_to_drop = ['oral', "ID"]
df_pred = df_pred.drop(columns=cols_to_drop)
df_pred

,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar
0,M,25,160,65,3.42,0.05,0.00,0.04,0.04,5.04,...,1.96,3.04,0.63,0.04,0.01,0.75,0.71,0.71,0,Y
1,M,30,180,80,3.46,0.04,0.01,0.04,0.04,5.38,...,1.50,4.21,0.59,0.04,0.04,0.79,1.13,1.33,0,N
2,M,55,165,60,3.42,0.00,0.01,0.04,0.04,5.25,...,2.13,2.04,0.63,0.04,0.01,1.08,1.29,2.00,1,Y
3,M,20,175,75,3.63,0.05,0.05,0.04,0.04,5.33,...,1.71,3.71,0.63,0.04,0.04,0.83,0.58,0.46,0,N
4,M,25,165,80,3.79,0.04,0.04,0.04,0.04,4.50,...,2.38,6.63,0.67,0.04,0.04,1.25,1.63,1.96,1,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,F,40,170,65,3.13,0.01,0.01,0.04,0.04,4.58,...,3.13,4.92,0.50,0.04,0.00,0.58,0.29,0.42,1,Y
5688,F,45,160,50,2.92,0.04,0.04,0.04,0.04,4.21,...,3.04,3.29,0.58,0.04,0.01,0.83,0.50,0.58,0,Y
5689,F,55,160,50,2.84,0.04,0.04,0.04,0.04,4.88,...,3.29,2.63,0.50,0.04,0.00,0.71,0.46,0.50,0,N
5690,M,60,165,60,3.25,0.01,0.04,0.04,0.04,5.54,...,2.00,6.08,0.59,0.04,0.00,0.83,0.79,0.75,0,N


In [7]:
df_pred["BMI"] = df_pred["weight(kg)"] / (df_pred["height(cm)"]/100) ** 2

In [8]:
cols_to_drop = [
    'height(cm)', 'weight(kg)', 'hearing(left)', 'hearing(right)',
]
df_pred = df_pred.drop(columns=cols_to_drop)
df_pred

,gender,age,waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,...,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar,BMI
0,M,25,3.42,0.05,0.00,5.04,3.13,4.00,5.46,2.29,...,3.04,0.63,0.04,0.01,0.75,0.71,0.71,0,Y,25.390625
1,M,30,3.46,0.04,0.01,5.38,3.25,4.17,7.92,10.96,...,4.21,0.59,0.04,0.04,0.79,1.13,1.33,0,N,24.691358
2,M,55,3.42,0.00,0.01,5.25,3.46,4.08,6.96,13.92,...,2.04,0.63,0.04,0.01,1.08,1.29,2.00,1,Y,22.038567
3,M,20,3.63,0.05,0.05,5.33,3.50,3.75,6.50,5.29,...,3.71,0.63,0.04,0.04,0.83,0.58,0.46,0,N,24.489796
4,M,25,3.79,0.04,0.04,4.50,3.00,4.17,10.46,7.25,...,6.63,0.67,0.04,0.04,1.25,1.63,1.96,1,Y,29.384757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,F,40,3.13,0.01,0.01,4.58,2.83,3.71,8.88,4.13,...,4.92,0.50,0.04,0.00,0.58,0.29,0.42,1,Y,22.491349
5688,F,45,2.92,0.04,0.04,4.21,2.58,3.71,6.92,2.88,...,3.29,0.58,0.04,0.01,0.83,0.50,0.58,0,Y,19.531250
5689,F,55,2.84,0.04,0.04,4.88,3.00,3.67,6.58,3.21,...,2.63,0.50,0.04,0.00,0.71,0.46,0.50,0,N,19.531250
5690,M,60,3.25,0.01,0.04,5.54,3.17,4.46,8.75,3.29,...,6.08,0.59,0.04,0.00,0.83,0.79,0.75,0,N,22.038567


In [9]:
df_pred = pd.get_dummies(df_pred, columns=['gender', 'tartar'], dtype=int)
df_pred

,age,waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,...,serum creatinine,AST,ALT,Gtp,dental caries,BMI,gender_F,gender_M,tartar_N,tartar_Y
0,25,3.42,0.05,0.00,5.04,3.13,4.00,5.46,2.29,1.96,...,0.01,0.75,0.71,0.71,0,25.390625,0,1,0,1
1,30,3.46,0.04,0.01,5.38,3.25,4.17,7.92,10.96,1.50,...,0.04,0.79,1.13,1.33,0,24.691358,0,1,1,0
2,55,3.42,0.00,0.01,5.25,3.46,4.08,6.96,13.92,2.13,...,0.01,1.08,1.29,2.00,1,22.038567,0,1,0,1
3,20,3.63,0.05,0.05,5.33,3.50,3.75,6.50,5.29,1.71,...,0.04,0.83,0.58,0.46,0,24.489796,0,1,1,0
4,25,3.79,0.04,0.04,4.50,3.00,4.17,10.46,7.25,2.38,...,0.04,1.25,1.63,1.96,1,29.384757,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,40,3.13,0.01,0.01,4.58,2.83,3.71,8.88,4.13,3.13,...,0.00,0.58,0.29,0.42,1,22.491349,1,0,0,1
5688,45,2.92,0.04,0.04,4.21,2.58,3.71,6.92,2.88,3.04,...,0.01,0.83,0.50,0.58,0,19.531250,1,0,0,1
5689,55,2.84,0.04,0.04,4.88,3.00,3.67,6.58,3.21,3.29,...,0.00,0.71,0.46,0.50,0,19.531250,1,0,1,0
5690,60,3.25,0.01,0.04,5.54,3.17,4.46,8.75,3.29,2.00,...,0.00,0.83,0.79,0.75,0,22.038567,0,1,1,0


Cargar modelo entrenado y hacer predicciones

In [10]:
import pickle

In [11]:
model = None

with open("./models/xgboost_smoking_model.pkl", "rb") as f:
  model = pickle.load(f)  

In [12]:
predictions = model.predict(df_pred)
predictions

array([0, 0, 1, ..., 0, 0, 1])

In [13]:
predictions_df = pd.DataFrame({
  "ID": ids,
  "smoking": predictions
})

predictions_df

,ID,smoking
0,27358,0
1,27364,0
2,27368,1
3,27378,0
4,27381,1
...,...,...
5687,55676,0
5688,55681,0
5689,55683,0
5690,55684,0


In [14]:
predictions_df.to_excel("./smoking_predictions.xlsx", index=False)